
<large>
<b>Build the Housing Adequacy Dataset (Quarterly)</b>
</large>




<large>
<b>Imports</b>
</large>


In [45]:


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns




<large>
<b>Context:</b>
The housing starts data is in SAAR = Seasonally Adjusted Annual Rate (seasonality has been statistically
removed by CMHC). Each monthly SAAR value is annualized — i.e.,
“If builders kept starting homes at this month’s pace for a full year, how many homes would be started in that year (in thousands)?”

When averaging the 3 months in a quarter, we get the <i>average annualized pace</i> for that quarter.
Because we’re using SAAR-based housing data, our quarterly averages are already seasonally adjusted.
We can safely compare them directly with population growth, but we should explicitly note that we are
using seasonally adjusted data.
</large>




<large>
<b>Read raw data (keep original paths; adjust locally if needed)</b>
</large>


In [46]:


housing_df = pd.read_csv('data/housing/housing_start_19902025.csv')
housing_df = housing_df.T
housing_df.head(5)
print(housing_df.columns.tolist())

population_df = pd.read_csv('data/housing/can_pop_19902025.csv')
population_df = population_df.T
population_df.head(10)


['Canada', 'Atlantic provinces', 'Newfoundland and Labrador', 'Prince Edward Island', 'Nova Scotia', 'New Brunswick', 'Quebec', 'Ontario', 'Prairie provinces', 'Manitoba', 'Saskatchewan', 'Alberta', 'British Columbia']


,Canada,Newfoundland and Labrador,Prince Edward Island,Nova Scotia,New Brunswick,Quebec,Ontario,Manitoba,Saskatchewan,Alberta,British Columbia,Yukon,Northwest Territories 5,Nunavut 5
Q1 1990,"27,463,550","576,278","130,336","907,394","737,307","6,955,119","10,189,985","1,102,752","1,011,429","2,520,056","3,247,419","27,605",..,..
Q2 1990,"27,567,161","577,167","130,468","908,444","738,744","6,973,762","10,238,884","1,103,935","1,009,952","2,532,621","3,266,888","27,824",..,..
Q3 1990,"27,691,138","577,368","130,404","910,451","740,156","6,996,986","10,295,832","1,105,421","1,007,727","2,547,788","3,292,111","27,957",..,..
Q4 1990,"27,807,591","577,113","130,367","911,749","741,981","7,019,039","10,344,678","1,105,098","1,004,214","2,563,142","3,322,896","28,023",..,..
Q1 1991,"27,854,861","577,377","130,477","912,792","743,210","7,026,241","10,355,101","1,106,196","1,002,651","2,572,947","3,339,935","28,223",..,..
Q2 1991,"27,928,837","578,397","130,453","912,889","744,203","7,044,395","10,385,937","1,107,757","1,002,876","2,580,625","3,352,585","28,494",..,..
Q3 1991,"28,037,420","579,644","130,369","914,969","745,567","7,067,396","10,431,316","1,109,604","1,002,713","2,592,306","3,373,787","28,871","38,724","22,154"
Q4 1991,"28,127,327","579,549","130,360","916,896","745,886","7,080,336","10,465,562","1,109,400","1,000,942","2,604,031","3,404,049","29,142","38,854","22,320"
Q1 1992,"28,181,477","579,425","130,604","917,302","746,571","7,082,645","10,488,022","1,109,978","1,001,136","2,611,786","3,423,217","29,239","39,055","22,497"
Q2 1992,"28,269,699","579,761","130,812","917,555","747,232","7,096,705","10,528,346","1,111,050","1,002,831","2,620,771","3,443,237","29,510","39,230","22,659"




<large>
<b>Assumptions & Format</b>
- Assume a constant average household size (AHS) = 2.5.
- Current data are in wide format; we'll convert to long.
</large>


In [48]:
print(population_df.columns.tolist())
provinces = {"Canada": 'can',
             'Newfoundland and Labrador': 'nl',
             'Prince Edward Island': 'pe',
             'Nova Scotia': 'ns',
             'New Brunswick': 'nb',
             'Quebec': 'qc',
             'Ontario': 'on',
             'Manitoba': 'mb',
             'Saskatchewan': 'sk',
             'Alberta': 'ab', 
             'British Columbia': 'bc'
             }


['Canada', 'Newfoundland and Labrador', 'Prince Edward Island', 'Nova Scotia', 'New Brunswick', 'Quebec', 'Ontario', 'Manitoba', 'Saskatchewan', 'Alberta', 'British Columbia', 'Yukon', 'Northwest Territories 5', 'Nunavut 5']




<large>
<b>Filter to provinces and rename columns to codes</b>
</large>




In [49]:

population = population_df.drop(
    columns=[c for c in population_df.columns if any(x in c for x in ["Yukon","Northwest","Nunavut","Atlantic","Prairie"])],
    errors="ignore"
)
population = population.rename(columns=provinces)
population.index

# For housing
housing = housing_df.drop(
    columns=[c for c in housing_df.columns if any(x in c for x in ["Yukon","Northwest","Nunavut","Atlantic","Prairie"])],
    errors="ignore"
)
housing = housing.rename(columns=provinces)
housing


,can,nl,pe,ns,nb,qc,on,mb,sk,ab,bc
Jan-90,276.428,4.811,0.804,5.840,1.847,62.026,118.510,9.536,1.484,25.399,46.171
Feb-90,255.893,6.115,1.022,5.610,5.341,77.397,77.476,5.469,1.219,20.735,55.509
Mar-90,254.693,6.737,1.469,4.833,7.719,64.967,94.289,4.401,1.486,24.131,44.661
Apr-90,215.859,2.944,0.419,8.486,3.362,44.666,72.744,4.199,1.773,24.813,52.453
May-90,214.111,3.422,0.391,4.208,2.419,45.994,78.534,4.357,1.991,24.311,48.484
...,...,...,...,...,...,...,...,...,...,...,...
Apr-25,282.679,2.100,2.232,13.665,4.089,57.322,70.410,6.728,7.800,63.583,54.750
May-25,283.294,1.599,0.594,9.640,8.638,67.485,72.711,9.007,5.663,68.939,39.018
Jun-25,283.914,1.677,0.743,9.314,9.365,58.830,62.624,5.728,4.893,63.565,67.175
Jul-25,293.537,1.720,2.904,17.662,9.785,63.773,77.637,6.070,5.143,51.843,57.000


<large>
<b>Parse monthly dates and sort </b>
</large>


In [50]:
housing.index = pd.to_datetime(housing.index, format='%b-%y')
housing = housing.sort_index()
housing
housing.index.is_monotonic_increasing


True



<large>
<b>Convert monthly SAAR to quarterly mean SAAR (demo with 'qc' first)</b>
The quarterly value is the mean of the three monthly SAAR observations in that quarter.
</large>




In [52]:
qc_series = housing.qc
type(qc_series.index)
qc_quarter = qc_series.resample('QE-DEC').mean()
qc_quarter.index

# 1990-03-31 -> 68.13 → average of Jan + Feb + Mar 1990 SAARs
# 1990-06-30 -> 48.775333 → average of Apr + May + Jun 1990 SAARs


DatetimeIndex(['1990-03-31', '1990-06-30', '1990-09-30', '1990-12-31',
               '1991-03-31', '1991-06-30', '1991-09-30', '1991-12-31',
               '1992-03-31', '1992-06-30',
               ...
               '2023-06-30', '2023-09-30', '2023-12-31', '2024-03-31',
               '2024-06-30', '2024-09-30', '2024-12-31', '2025-03-31',
               '2025-06-30', '2025-09-30'],
              dtype='datetime64[ns]', length=143, freq='QE-DEC')



<large>
<b>Use PeriodIndex (Q) for quarter labels like 1990Q1</b>
</large>




In [53]:
qc_quarter.index = qc_quarter.index.to_period('Q')
qc_quarter.index


PeriodIndex(['1990Q1', '1990Q2', '1990Q3', '1990Q4', '1991Q1', '1991Q2',
             '1991Q3', '1991Q4', '1992Q1', '1992Q2',
             ...
             '2023Q2', '2023Q3', '2023Q4', '2024Q1', '2024Q2', '2024Q3',
             '2024Q4', '2025Q1', '2025Q2', '2025Q3'],
            dtype='period[Q-DEC]', length=143)



<large>
<b>Quarterly SAAR for all provinces; reshape to long</b>
</large>




In [54]:
housing_quarter = housing.resample('Q-DEC').mean()
housing_quarter.index = housing_quarter.index.to_period('Q')

housing_long = housing_quarter.stack().reset_index()
housing_long.columns = ['quarter', 'province', 'starts_saar']
housing_long = housing_long.set_index('quarter').reset_index()
housing_long


/var/folders/8y/55dw_cwj3hv6wylylp7fwhb80000gn/T/ipykernel_49661/3894633765.py:1: FutureWarning: 'Q-DEC' is deprecated and will be removed in a future version, please use 'QE-DEC' instead.
  housing_quarter = housing.resample('Q-DEC').mean()


,quarter,province,starts_saar
0,1990Q1,can,262.338000
1,1990Q1,nl,5.887667
2,1990Q1,pe,1.098333
3,1990Q1,ns,5.427667
4,1990Q1,nb,4.969000
...,...,...,...
1568,2025Q3,on,68.557500
1569,2025Q3,mb,7.775500
1570,2025Q3,sk,5.009000
1571,2025Q3,ab,49.756500




<large>
<b>Tidy population table (quarter strings → PeriodIndex)</b>
</large>




In [55]:
population
population.index = population.index.str.replace(r'Q(\d)\s+(\d{4})', r'\2Q\1', regex=True)
population.index = pd.PeriodIndex(population.index, freq='Q')

population.index
population_long = population.stack().reset_index()
population_long.columns = ['quarter', 'province', 'population']
population.head()
population.index
population_long
population_long['population'] = population_long['population'].str.replace(',','').astype(float)
population_long


,quarter,province,population
0,1990Q1,can,27463550.0
1,1990Q1,nl,576278.0
2,1990Q1,pe,130336.0
3,1990Q1,ns,907394.0
4,1990Q1,nb,737307.0
...,...,...,...
1568,2025Q3,on,16258260.0
1569,2025Q3,mb,1509702.0
1570,2025Q3,sk,1266959.0
1571,2025Q3,ab,5029346.0




<large>
<b>Cleaned tables to work with:</b> <code>population_long</code>, <code>housing_long</code>
</large>




In [56]:
print(population_long.isnull().sum())
print(housing_long.isnull().sum())


quarter       0
province      0
population    0
dtype: int64
quarter        0
province       0
starts_saar    0
dtype: int64




<large>
<b>Inner-join on (quarter, province)</b>
</large>




In [57]:
df_final = pd.merge(population_long, housing_long, on=['quarter', 'province'], how='inner')
df = df_final.copy()
df


,quarter,province,population,starts_saar
0,1990Q1,can,27463550.0,262.338000
1,1990Q1,nl,576278.0,5.887667
2,1990Q1,pe,130336.0,1.098333
3,1990Q1,ns,907394.0,5.427667
4,1990Q1,nb,737307.0,4.969000
...,...,...,...,...
1568,2025Q3,on,16258260.0,68.557500
1569,2025Q3,mb,1509702.0,7.775500
1570,2025Q3,sk,1266959.0,5.009000
1571,2025Q3,ab,5029346.0,49.756500




<large>
<b>Convert SAAR to quarterly units (de-annualize) and to dwellings</b>
- SAAR is in thousands of dwellings per year.
- Quarterly rate = SAAR / 4 (thousands per quarter).
- Dwellings per quarter = (SAAR / 4) * 1000.
</large>




In [58]:
df['starts_saar_q'] = df['starts_saar']/4
df['dwelling_starts'] = df['starts_saar_q'] * 1000




<large>
<b>Compute HAI</b>
HAI = starts / needed, with needed = Δpop / 2.5. We cap spikes with an IQR rule per province.
</large>


In [59]:


df_cp = df.copy()
df.isnull().sum()

# Housing Adequacy Index (HAI):
# HAI = starts/needed, if <1: inadequate, >1: surplus

def cap_spikes(x):
    Q1, Q3 = x.quantile([0.25, 0.75])
    IQR = Q3 - Q1
    spikes = (x < Q1 - 1.5*IQR) | (x > Q3 + 1.5*IQR)
    cap = x[~spikes].max()
    return np.where(spikes, cap, x)


df = df_cp.copy()
df['pop_change_q'] = df.groupby('province')['population'].diff()
df['needed_units_q'] = df['pop_change_q']/2.5  # considering an average household size = 2.5
df.loc[df['pop_change_q'] <= 0, 'needed_units_q'] = 0  # ignore population decline

# df['hai'] = df['dwelling_starts']/df['needed_units_q']
df['hai_init'] = np.where(df['needed_units_q'] > 0,
                     df['dwelling_starts'] / df['needed_units_q'],
                     0.0)


df['hai'] = df.groupby('province')['hai_init'].transform(cap_spikes)
df = df.drop(columns=['hai_init'])




<large>
<b>Basic QA</b>
</large>




In [60]:
df.isnull().sum()
df.describe()
# Sanity check
_df_sk = df.query("province=='sk'")[['quarter','population','hai']]
_df_sk


,quarter,population,hai
8,1990Q1,1011429.0,0.000000
19,1990Q2,1009952.0,0.000000
30,1990Q3,1007727.0,0.000000
41,1990Q4,1004214.0,0.000000
52,1991Q1,1002651.0,0.000000
...,...,...,...
1526,2024Q3,1247868.0,0.385115
1537,2024Q4,1256983.0,0.314774
1548,2025Q1,1261524.0,0.839802
1559,2025Q2,1264537.0,1.269222




<large>
<b>Data dictionary (from original text)</b>
</large>





| <b>Column</b> | <b>Meaning</b> | <b>Units</b> |
|:------------|:-------------|:-----------|
| <code>quarter</code> | Time (PeriodIndex, quarterly) | Q-DEC |
| <code>province</code> | Province code | — |
| <code>population</code> | Total population | persons |
| <code>starts_saar</code> | CMHC seasonally adjusted annual rate | 1000s dwellings/year |
| <code>starts_saar_q</code> | Approx. actual housing starts | 1000s dwellings/quarter |
| <code>dwelling_starts</code> | Approximate number of dwellings started per quarter (converted from SAAR, which is in thousands of dwellings per year) | dwellings/quarter |
| <code>pop_change_q</code> | Quarterly population change | persons |
| <code>needed_units_q</code> | Required homes = Δpop / 2.5 | dwellings (not ×1000) |
| <code>hai</code> | Housing Adequacy Index = starts_saar_q / needed_units_q | unitless ratio |





<large>
<b>Quick checks before proceeding</b>
</large>




In [62]:
df[df["pop_change_q"].isna()].quarter.unique()  # to check that NaN values are only in 1990Q1

df.query("pop_change_q < -100000")  # checking for wild negative growth

# HAI values become infinite when quarterly population change (dpop_q) is zero or negative,
# since the denominator (needed_units_q) becomes 0 or < 0. These cases are excluded (HAI = NaN)
# as they don’t represent meaningful housing adequacy ratios. Thus, NaN values now appear in the dataset.
# df.loc[df["needed_units_q"] == 0, "hai"] = np.nan  # defining HAI only if the population is not changing

(df.groupby('province')['hai'].mean().sort_values())

df.query("province=='sk'")[["quarter","pop_change_q","population", 'hai']].tail(30)

# df.query("province=='qc'").set_index("quarter")[['hai']].plot()

df.isnull().sum()


quarter             0
province            0
population          0
starts_saar         0
starts_saar_q       0
dwelling_starts     0
pop_change_q       11
needed_units_q     11
hai                 0
dtype: int64



<large>
<b>Save final dataset</b>
</large>




In [63]:
df.to_csv("data/housing_adequacy_dataset.csv", index=False)




<large>
<b>Modeling role of each column (from original text)</b>
</large>





| <b>Column</b> | <b>Role in Modeling</b> | <b>Why it Matters</b> |
|:------------|:---------------------|:--------------------|
| <code>quarter</code> | <b>Time index</b> | Used to split train/test chronologically; can extract features like year or quarter of year. |
| <code>province</code> | <b>Grouping key</b> | Each province can have its own model; enables province-wise analysis and comparisons. |
| <code>population</code> | <b>Level indicator</b> | Captures structural scale — larger populations imply higher housing demand baseline. |
| <code>pop_change_q</code> | <b>Primary driver</b> | Reflects short-term demographic shifts; key predictor of housing demand. |
| <code>needed_units_q</code> | <b>Derived demand feature</b> | Converts population change into estimated housing need (Δpop / 2.5). |
| <code>starts_saar_q</code> | <b>Target variable</b> | Quarterly housing starts to be forecasted; main outcome variable. |
| <code>starts_saar</code> | <b>Reference variable</b> | Original CMHC SAAR values (annualized); helps check scaling and realism. |
| <code>hai</code> | <b>Diagnostic variable</b> | Evaluates adequacy of supply vs. demand; used for post-model analysis and validation. |
| <code>starts_saar_q_lag1</code>, <code>starts_saar_q_lag4</code> | <b>Temporal features</b> | Capture persistence and quarterly/annual seasonality in housing starts. |





<large>
<b>Final sanity summaries</b>
</large>


In [64]:


df[["dwelling_starts", "needed_units_q", "hai"]].describe().T

df.loc[df["needed_units_q"].abs() < 10, ["quarter", "province", "needed_units_q", "hai"]]


,quarter,province,needed_units_q,hai
19,1990Q2,sk,0.0,0.0
24,1990Q3,pe,0.0,0.0
30,1990Q3,sk,0.0,0.0
34,1990Q4,nl,0.0,0.0
35,1990Q4,pe,0.0,0.0
...,...,...,...,...
1394,2021Q3,sk,0.0,0.0
1542,2025Q1,pe,0.0,0.0
1556,2025Q2,qc,0.0,0.0
1561,2025Q2,bc,0.0,0.0
